In [7]:
!pip install ultralytics==8.2.51
!pip install opencv-python
!pip install shapely>=2.0.0
!pip install lapx>=0.5.2
!pip install gradio
!pip install gdown

In [8]:
import cv2
import gradio as gr
import gdown
from ultralytics import YOLO, solutions

css = '''
.gradio-container{max-width: 600px !important}
h1{text-align:center}
footer {
    visibility: hidden
}
'''
js_func = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

# Download YOLO models from Google Drive
yolo_model_url = "https://drive.google.com/uc?id=1-6592iHHUcSBJ6mvzkARykGH9HkIJG1d"
pose_model_url = "https://drive.google.com/uc?id=1hg2p87gTVxYhup7vicgJNYlFaRcnSJHL"

gdown.download(yolo_model_url, "yolov8x.pt", quiet=False)
gdown.download(pose_model_url, "yolov8x-pose-p6.pt", quiet=False)

# Load YOLO models
yolo_model = YOLO("yolov8x.pt")
pose_model = YOLO("yolov8x-pose-p6.pt")
names = yolo_model.model.names

# Initialize Solutions
speed_obj = solutions.SpeedEstimator(
    reg_pts=[(0, 360), (1280, 360)],
    names=names,
    view_img=False,
)

region_points = [(20, 400), (1080, 404), (1080, 360), (20, 360)]
counter = solutions.ObjectCounter(
    view_img=False,
    reg_pts=region_points,
    classes_names=yolo_model.names,
    draw_tracks=True,
    line_thickness=2,
)

dist_obj = solutions.DistanceCalculation(names=names, view_img=False)

gym_object = solutions.AIGym(
    line_thickness=2,
    view_img=False,
    pose_type="pushup",
    kpts_to_check=[6, 8, 10],
)

heatmap_obj = solutions.Heatmap(
    colormap=cv2.COLORMAP_PARULA,
    view_img=False,
    shape="circle",
    classes_names=yolo_model.names,
)

def process_video(video_path, function):
    cap = cv2.VideoCapture(video_path)
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    output_path = f"{function}_output.avi"
    video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            break

        if function == "Speed Estimation":
            tracks = yolo_model.track(im0, persist=True, show=False)
            im0 = speed_obj.estimate_speed(im0, tracks)
        elif function == "Object Counting":
            tracks = yolo_model.track(im0, persist=True, show=False)
            im0 = counter.start_counting(im0, tracks)
        elif function == "Distance Calculation":
            tracks = yolo_model.track(im0, persist=True, show=False)
            im0 = dist_obj.start_process(im0, tracks)
        elif function == "Workout Monitoring":
            results = pose_model.track(im0, verbose=False)
            im0 = gym_object.start_counting(im0, results)
        elif function == "Heatmaps":
            tracks = yolo_model.track(im0, persist=True, show=False)
            im0 = heatmap_obj.generate_heatmap(im0, tracks)

        video_writer.write(im0)

    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()

    return output_path

def gradio_interface(video, function):
    processed_video_path = process_video(video, function)
    return processed_video_path

# Create Gradio Blocks Interface
with gr.Blocks(css=css, js=js_func) as demo:
    gr.Markdown("# YOLOv8 Ultralytics Solutions")

    with gr.Row():
        video_input = gr.Video()
        function_selector = gr.Dropdown(
            choices=["Speed Estimation", "Object Counting", "Distance Calculation", "Workout Monitoring", "Heatmaps"],
            label="Select Function",
            value="Speed Estimation",
        )

    output_video = gr.Video()
    process_button = gr.Button("Process Video")

    process_button.click(fn=gradio_interface, inputs=[video_input, function_selector], outputs=output_video)

    gr.Markdown("⚠️ The videos that are 30 seconds or less will avoid time consumption issues and provide better accuracy.")

demo.launch(share=True)

Downloading...
From (original): https://drive.google.com/uc?id=1-6592iHHUcSBJ6mvzkARykGH9HkIJG1d
From (redirected): https://drive.google.com/uc?id=1-6592iHHUcSBJ6mvzkARykGH9HkIJG1d&confirm=t&uuid=fef25dcd-cf95-4389-a5d9-cc0209217bfb
To: /content/yolov8x.pt
100%|██████████| 137M/137M [00:00<00:00, 149MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1hg2p87gTVxYhup7vicgJNYlFaRcnSJHL
From (redirected): https://drive.google.com/uc?id=1hg2p87gTVxYhup7vicgJNYlFaRcnSJHL&confirm=t&uuid=b885d05d-1109-4a82-a0be-ba182253c02a
To: /content/yolov8x-pose-p6.pt
100%|██████████| 199M/199M [00:04<00:00, 49.0MB/s]


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

None
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9bfa56b234ccc49c7c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
